In [ ]:
! pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=f7f10c4d01e8b5095a361a95d948562206a648af3b7c20ff14d6aac0116317af
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
# connect to google drive to run on colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
%cd /content/drive/MyDrive/CSE6240/notebooks

/content/drive/.shortcut-targets-by-id/1y1Lvxa-gIYQlLQs7Dzoz9XVXO1BPTpP5/CSE6240/notebooks


In [12]:
df_product = pd.read_csv('../data/raw/products.csv')

In [13]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [14]:
df_product.shape

(49688, 4)

In [15]:
df_product["product_id"].max()

49688

In [16]:
df_product["product_name"].isna().sum()

0

In [20]:
# use sentence transformer to get embedding for product name
# define a function to get embedding for one product name

st_model = SentenceTransformer('all-mpnet-base-v2', device='cuda')

In [21]:
emb_dim = st_model.encode("Banana").shape[0]
prod_emb = np.zeros((df_product["product_id"].max()+1, emb_dim))


In [22]:
prod_emb[1:,:] = st_model.encode(df_product["product_name"])

In [28]:
np.save("../data/feature_emb/products.npy", prod_emb)

In [29]:
for file_name in ("aisles", "departments"):
    df_ = pd.read_csv(f'../data/raw/{file_name}.csv')
    emb_ = np.zeros((df_.iloc[:,0].max()+1, emb_dim))
    emb_[1:,:] = st_model.encode(df_.iloc[:,1])
    np.save(f"../data/feature_emb/{file_name}.npy", emb_)